In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
# Generate synthetic data
num_nodes = 10
num_edges = 20

# Generate random adjacency matrix
adjacency_matrix = np.random.randint(0, 2, size=(num_nodes, num_nodes))

# Ensure symmetric adjacency
adjacency_matrix = np.maximum(adjacency_matrix, adjacency_matrix.T)

# Create random edge features
edge_features = np.random.rand(num_edges, 1)

# Label each edge as 'good' or 'bad' (binary classification)
edge_labels = np.random.choice(['good', 'bad'], size=num_edges)

# Map labels to integers (0 for 'bad', 1 for 'good')
edge_labels = np.array([0 if label == 'bad' else 1 for label in edge_labels])

# Convert to PyTorch tensors
adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32)
edge_features = torch.tensor(edge_features, dtype=torch.float32)
edge_labels = torch.tensor(edge_labels, dtype=torch.float32)


In [8]:
# Generate synthetic data
num_nodes = 10
num_edges = 20

# Generate random adjacency matrix
adjacency_matrix = torch.tensor(np.random.randint(0, 2, size=(num_nodes, num_nodes)), dtype=torch.float32)

# Ensure symmetric adjacency
adjacency_matrix = torch.maximum(adjacency_matrix, adjacency_matrix.T)

# Get indices of non-zero elements in the adjacency matrix
edge_indices = torch.nonzero(adjacency_matrix, as_tuple=False)

# Create random edge features and labels based on the number of edges
edge_features = torch.randn(num_edges, 1)
edge_labels = torch.randint(0, 2, (num_edges,), dtype=torch.float32)

print("Adjacency Matrix Shape:", adjacency_matrix.shape)
print("Edge Features Shape:", edge_features.shape)
print("Edge Labels Shape:", edge_labels.shape)


Adjacency Matrix Shape: torch.Size([10, 10])
Edge Features Shape: torch.Size([20, 1])
Edge Labels Shape: torch.Size([20])


In [10]:
import keras
model = keras.Sequential()
model.add(keras.layers.Embedding(1000, 64))
# The model will take as input an integer matrix of size (batch,
# input_length), and the largest integer (i.e. word index) in the input
# should be no larger than 999 (vocabulary size).
# Now model.output_shape is (None, 10, 64), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(32, 10))
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)




1/1 [==============================] - 0s 182ms/step
(32, 10, 64)


In [3]:
class EGATLayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(EGATLayer, self).__init__()
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(2*out_dim, 1, bias=False)
        self.relu = nn.ReLU()
        
    def forward(self, x, adjacency_matrix):
        h = self.fc(x)
        batch_size, num_nodes, _ = h.size()
        
        # Pairwise node attention
        attn_input = torch.cat([h.unsqueeze(2).expand(-1, -1, num_nodes, -1),
                                h.unsqueeze(1).expand(-1, num_nodes, -1, -1)], dim=3)
        attn_scores = self.attn_fc(attn_input).squeeze(3)
        attn_scores = torch.where(adjacency_matrix.unsqueeze(0).expand(batch_size, -1, -1) > 0, attn_scores, torch.tensor(-9e15))
        attn_scores = torch.softmax(attn_scores, dim=2)
        
        # Weighted sum of neighbors' features
        h_prime = torch.matmul(attn_scores, h)
        
        return self.relu(h_prime)

class EGAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(EGAT, self).__init__()
        self.egat_layer = EGATLayer(in_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, out_dim)
        
    def forward(self, x, adjacency_matrix):
        h = self.egat_layer(x, adjacency_matrix)
        out = self.fc(h.mean(dim=1))
        return out


In [5]:
print("Adjacency Matrix Shape:", adjacency_matrix.shape)
print("Edge Features Shape:", edge_features.shape)
print("Edge Labels Shape:", edge_labels.shape)

Adjacency Matrix Shape: torch.Size([10, 10])
Edge Features Shape: torch.Size([20, 1])
Edge Labels Shape: torch.Size([20])


In [4]:
# Split data into training and testing sets
adj_train, adj_test, features_train, features_test, labels_train, labels_test = train_test_split(
    adjacency_matrix, edge_features, edge_labels, test_size=0.2, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [10, 20, 20]

In [ ]:
# Instantiate the model
model = EGAT(in_dim=1, hidden_dim=32, out_dim=1)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(features_train.unsqueeze(0), adj_train.unsqueeze(0))
    loss = criterion(output.squeeze(), labels_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluate on test set
with torch.no_grad():
    test_output = model(features_test.unsqueeze(0), adj_test.unsqueeze(0))
    predicted_labels = torch.round(torch.sigmoid(test_output))
    accuracy = (predicted_labels.squeeze() == labels_test).float().mean()
    print(f'Accuracy on test set: {accuracy.item()}')
